In [1]:
# This cell is used to setup the Python environment.  It only needs to run once.

# Import the slip report code from slip_report_funcs.py in this directory 
import slip_report_funcs as sr
sr.Setup()

2026-02-06 12:31:30,395 (INFO) NumExpr defaulting to 10 threads.


Loading BokehJS ...

2026-02-06 12:31:30,693 (INFO) Plotting function loaded.
2026-02-06 12:31:30,693 (INFO) Slip report initialized.


In [2]:
# This cell sets up the report criteria.  When you alter any of these values, re-run the cell.

# Telescope, specify either 1 or 2
telescope = 1

# Telescope azimuth rotation.  The default graph plots AZ of 0 pointing to the right.  
# Use this to rotate it to match the typical diagrams of the building.
azimuth_rotate = -45

# Time range.  Choices are:
#   days_ago = 15    (Look backwards from today, this number of days.)
#       OR
#   days_ago = 0     (Set this to zero to use a specific start and end date.)
#   start_date = '20251217'
#   end_date = '20260122'

days_ago = 0
# OR...
start_date = '20260205'
end_date = '20260207'


In [3]:
# This cell computes the the time range of your query and plots it below.

if days_ago > 0:

    today = sr.datetime.date.today()
    dt_today = sr.datetime.datetime(today.year, today.month, today.day, 0, 0)

    t0 = dt_today - sr.datetime.timedelta(days=days_ago)
    t1 = dt_today + sr.datetime.timedelta(days=1)

else:
    import dateutil.parser
    t0 = dateutil.parser.parse(start_date)
    t1 = dateutil.parser.parse(end_date) 

# Get the data for the range we are looking at
all_slips = sr.GetAllSlips(telescope, t0, t1)
reduced = sr.ReduceSlipData(all_slips)
sr.PlotSlips(reduced, top=azimuth_rotate)


2026-02-06 12:31:33,188 (INFO) Motor slip analysis for 1: 2026-02-05 00:00:00 to 2026-02-07 00:00:00
2026-02-06 12:31:35,298 (WARNING) No motor slips found for k1:dcs:axe:az:mtr1ASlip at 2026-02-05T00:00:00-10:00
2026-02-06 12:31:37,399 (WARNING) No motor slips found for k1:dcs:axe:az:mtr1BSlip at 2026-02-05T00:00:00-10:00
2026-02-06 12:31:39,469 (DEBUG) Starting new HTTP connection (1): k1epicsgateway:17668
2026-02-06 12:31:40,264 (DEBUG) http://k1epicsgateway:17668 "POST /retrieval/data/getDataAtTime.json?at=2026-02-06T07%3A49%3A44.076903105-00%3A00 HTTP/1.1" 200 197
2026-02-06 12:31:42,373 (WARNING) No motor slips found for k1:dcs:axe:az:mtr2BSlip at 2026-02-05T00:00:00-10:00
2026-02-06 12:31:44,421 (DEBUG) Starting new HTTP connection (1): k1epicsgateway:17668
2026-02-06 12:31:45,197 (DEBUG) http://k1epicsgateway:17668 "POST /retrieval/data/getDataAtTime.json?at=2026-02-06T14%3A35%3A39.077113628-00%3A00 HTTP/1.1" 200 196
2026-02-06 12:31:47,262 (WARNING) No motor slips found for k1

In [ ]:
# Run this cell to re-plot the retrieved data
sr.PlotSlips(reduced, flip=True, top=-45)

In [ ]:
# Run this cell to plot demo data, no retrieval needed
sr.PlotSlips(sr.demo_data, flip=True, top=azimuth_rotate)